# December 02, 2025

https://adventofcode.com/2025/day/2

In [ ]:
from math import log10, ceil, floor
import re

In [18]:
def parse_input(text):
    specs = text.split(",")
    return [ [int(x) for x in y.split("-")] for y in specs ]

In [19]:
text = f'''11-22,95-115,998-1012,1188511880-1188511890,222220-222224,
1698522-1698528,446443-446449,38593856-38593862,565653-565659,
824824821-824824827,2121212118-2121212124'''

text = "".join(text.split("\n"))
test = parse_input(text)

In [ ]:
fn = "../data/2025/02.txt"
with open(fn, "r") as file:
    text = file.readlines()
text = [line.strip() for line in text]
puzz = parse_input(text[0])

# Part 1

In [ ]:
def ndig( x ):
    '''number of digits in x'''

    # technically, we won't see 0, but just in cases
    if x == 0:
        return 1
    
    return ceil(log10(x+.001))

def create_sub_ranges( x0, x1 ):
    '''separate range [x0,x1] into a list of subranges each with consistent num digits'''
    len0 = ndig(x0)
    len1 = ndig(x1)

    if len0 == len1:
        return [[x0, x1]]
    
    ranges = [ [x0, 10**len0-1] ]
    
    for pwr in range(len0, len1-1):
        ranges.append( [10**pwr, 10**(pwr+1)-1] )

    ranges.append( [10**(len1-1), x1] )

    return ranges

def num_halves( x ):
    '''return the first n/2 digits of x and the last n/2 digits'''
    # Assume even number of digits
    nd = ndig( x )

    front = floor( x / 10**(nd/2) )
    back = int( x - front * 10**(nd/2) ) 
    
    return front, back

def sum_invalid_ids( x0, x1 ):
    '''sum all numbers in [x0, x1] that are the same substring repeated 1 time'''

    sub_ranges = create_sub_ranges( x0, x1 )
    tot = 0

    for s in sub_ranges:
        nd = ndig(s[0])
        if nd % 2 == 1:
            continue

        s0a, s0b = num_halves( s[0] )
        if s0b <= s0a:
            # first invalid is s0as0a bc it's larger than s0as0b
            minx = s0a
        else:
            # first invalid is cat(s0a+1, s0a+1)
            minx = s0a+1

        s1a, s1b = num_halves( s[1] )
        if s1b >= s1a:
            # last invalid number is s1as1a bc it's smaller than s1as1b
            maxx = s1a
        else:
            # last invalid number is cat(s1a-1, s1a-1)
            maxx = s1a-1

        if minx > maxx:
            # example 1234 — 1300
            # first invalid number after 1234 would be 1313, which is out of range
            tot += 0
        
        
        tot += (minx + maxx) / 2 * (maxx-minx+1) * (10**(nd/2) + 1)

    return tot


In [7]:
print( create_sub_ranges( 9, 1234) )
print( create_sub_ranges( 10, 1000) )

[[9, 9], [10, 99], [100, 999], [1000, 1234]]
[[10, 99], [100, 999], [1000, 1000]]


In [23]:
def part1( puzz ):
    tot = 0
    for r in puzz:
            tot += sum_invalid_ids( r[0], r[1] )
    
    return tot

In [26]:
part1( test )

1227775554.0

In [27]:
part1( puzz )

19128774598.0

# Part 2

In [24]:
def sum_invalid_strings2( lo, hi ):
    cnt = 0
    for x in range(lo, hi+1):
        cnt += 0 if re.fullmatch(r'(.+)\1+', str(x)) is None else x
    return cnt

def part2( puzz, verbose = False ):
    tot = 0
    for r in puzz:
            x = sum_invalid_strings2( r[0], r[1] )
            if verbose:
                 print(r, "-->", x)
            tot += x
    
    return tot

In [25]:
part2( test, True )

[11, 22] --> 33
[95, 115] --> 210
[998, 1012] --> 2009
[1188511880, 1188511890] --> 1188511885
[222220, 222224] --> 222222
[1698522, 1698528] --> 0
[446443, 446449] --> 446446
[38593856, 38593862] --> 38593859
[565653, 565659] --> 565656
[824824821, 824824827] --> 824824824
[2121212118, 2121212124] --> 2121212121


4174379265

In [26]:
part2( puzz )

21932258645

In [ ]:
def next_invalid_substring( x, L, up ):
    '''find the next invalid id that repeats a length-L sequence
    if up=True, return the next highest, otherwise return the next lowest
    only returns the length-L substring, not the actual id'''

    nd = ndig( x )

    # edge case, if nd is not a multiple of L, then no invalid ids are possible
    # or if L is more than half the length of x
    if nd % L > 0 or L > nd/2:
        return None
    
    # get the first L digits and the next L digits
    # if these are equal, get the NEXT L digits
    # keep going until we find the next unequal substring
    # or we run out of digits, so x= LLLLLL
    pwr = nd - L
    firstL = int( x / 10 ** pwr )
    tmp = x
    for i in range( L, nd, L ):
        tmp -= firstL * 10 ** (nd-i)
        nextL = int( tmp / 10 ** (nd-i-L) )
        if nextL != firstL:
            break

    if up:
        sub_seq = firstL + int(nextL > firstL)
    else:
        sub_seq = firstL - int(nextL < firstL)

    return sub_seq

def sum_invalid_ids2( x0, x1 ):
    sub_ranges = create_sub_ranges( x0, x1 )

    tot = 0
    for s in sub_ranges:
        nd = ndig(s[0])
        for L in range(1, int(nd/2)+1):
            # Get the starting and ending substrings
            minx = next_invalid_substring( s[0], L, up=True )
            if minx is None:
                continue
            maxx = next_invalid_substring( s[1], L, up=False )
            
            if maxx < minx:
                continue

            avg = ( minx + maxx ) / 2
            area = int( avg * (maxx - minx + 1) )

            sum_powers = sum( [10**p for p in range(0, nd, L)] )
           # print(avg, area, sum_powers)
            tot += area * sum_powers
    return tot

In [69]:
def part2( puzz ):
    tot = 0
    for r in puzz:
            x = sum_invalid_ids2( r[0], r[1] )
            print(x)
            tot += x
    
    return tot

In [97]:
next_invalid_substring(1188511880, 1, False)

(1, 8)

In [100]:
sum_invalid_ids2( test[3][0], test[3][1])

11885.0 11885 100001


1188511885

In [103]:
part2( test )

33
210
2009
1188511885
666666
0
446446
38593859
565656
824824824
2121212121


4174823709